In [1]:
import numpy as np
import os

from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector
from tensorflow_hub.module_v2 import load

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')
print(tf.__version__)

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

c:\Users\Joshua\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Users\Joshua\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.10.0 and strictly below 2.13.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.9.0 and is not supported. 
Some things might work, some things might not.
If you were to encount

2.9.0


In [2]:
train_ds = object_detector.DataLoader.from_pascal_voc(
    './project-6-at-2023-04-30-00-12-74270469/train',
    './project-6-at-2023-04-30-00-12-74270469/train',
    ['Counted_Objects', 'Not_Counted_Objects']
)

val_ds = object_detector.DataLoader.from_pascal_voc(
    './project-6-at-2023-04-30-00-12-74270469/validate',
    './project-6-at-2023-04-30-00-12-74270469/validate',
    ['Counted_Objects', 'Not_Counted_Objects']
)

In [3]:
print(train_ds.size)
print(val_ds.size)

108
27


In [4]:
#spec = model_spec.get('efficientdet_lite4')

spec = object_detector.EfficientDetSpec(
  model_name='efficientdet-lite4', 
  uri='https://tfhub.dev/tensorflow/efficientdet/lite4/feature-vector/2', 
  hparams={'max_instances_per_image': 8000})

In [5]:
model = object_detector.create(train_ds, model_spec=spec, batch_size=16, train_whole_model=True, epochs=200, validation_data=val_ds)

Epoch 1/200
6/6 [==============================] - 294s 39s/step - det_loss: 1.7722 - cls_loss: 1.1201 - box_loss: 0.0130 - reg_l2_loss: 0.1078 - loss: 1.8800 - learning_rate: 0.0137 - gradient_norm: 0.5092 - val_det_loss: 1.7311 - val_cls_loss: 1.1096 - val_box_loss: 0.0124 - val_reg_l2_loss: 0.1078 - val_loss: 1.8389
Epoch 2/200
6/6 [==============================] - 218s 37s/step - det_loss: 1.7076 - cls_loss: 1.0854 - box_loss: 0.0124 - reg_l2_loss: 0.1078 - loss: 1.8154 - learning_rate: 0.0200 - gradient_norm: 0.5219 - val_det_loss: 1.6102 - val_cls_loss: 1.0468 - val_box_loss: 0.0113 - val_reg_l2_loss: 0.1078 - val_loss: 1.7180
Epoch 3/200
6/6 [==============================] - 217s 37s/step - det_loss: 1.5623 - cls_loss: 0.9952 - box_loss: 0.0113 - reg_l2_loss: 0.1078 - loss: 1.6700 - learning_rate: 0.0200 - gradient_norm: 0.7419 - val_det_loss: 1.3644 - val_cls_loss: 0.8681 - val_box_loss: 0.0099 - val_reg_l2_loss: 0.1078 - val_loss: 1.4722
Epoch 4/200
6/6 [====================

In [9]:
model.evaluate(train_ds)

2/2 [==============================] - 68s 24s/step



{'AP': 0.6506272,
 'AP50': 0.82872033,
 'AP75': 0.7702021,
 'APs': 0.0,
 'APm': 0.0048326263,
 'APl': 0.6656851,
 'ARmax1': 0.18150418,
 'ARmax10': 0.49503738,
 'ARmax100': 0.7088769,
 'ARs': 0.0,
 'ARm': 0.008421052,
 'ARl': 0.725424,
 'AP_/Counted_Objects': 0.6188632,
 'AP_/Not_Counted_Objects': 0.68239117}

In [10]:
model.evaluate(val_ds)

1/1 [==============================] - 21s 21s/step



{'AP': 0.44312027,
 'AP50': 0.6468589,
 'AP75': 0.52726954,
 'APs': 0.0,
 'APm': 0.01980198,
 'APl': 0.44669285,
 'ARmax1': 0.14002526,
 'ARmax10': 0.36950272,
 'ARmax100': 0.5398585,
 'ARs': 0.0,
 'ARm': 0.018181818,
 'ARl': 0.54415184,
 'AP_/Counted_Objects': 0.40821844,
 'AP_/Not_Counted_Objects': 0.47802213}

In [7]:
model.export(export_dir='../obj_model_training/',tflite_filename='efd_4_uc_ma.tflite')